In [ ]:
using QuantumOptics
using PyPlot
using CorrelationExpansion
const ce = CorrelationExpansion;

In [ ]:
xmin = -6
xmax = 6
Npoints = 100
L = xmax - xmin
dx = L/Npoints
xpoints = [xmin:dx:xmax-dx;]

index_coupling = Int(floor(Npoints/2))

g = 1/2dx
g_atom = 0.4g;

In [ ]:
b_atom = NLevelBasis(2)
t21 = transition(b_atom, 2, 1)
t12 = transition(b_atom, 1, 2)

b_fock = FockBasis(1)
a = destroy(b_fock)
at = create(b_fock)
n = number(b_fock)

b_space = tensor([b_fock for i=1:Npoints]...)
b = b_atom ⊗ b_space

H = LazyTensor[]
for i=2:Npoints
    push!(H, g*LazyTensor(b, [i, i+1], [at, a]))
    push!(H, g*LazyTensor(b, [i, i+1], [a, at]))
    
end

push!(H, g_atom*LazyTensor(b, [1, index_coupling+1], [t21, a]))
push!(H, g_atom*LazyTensor(b, [1, index_coupling+1], [t12, at]))

H = LazySum(H...)
J = LazyTensor[];

In [ ]:
# S_space = [[i, i+1] for i=2:Npoints-1]
# S = [[1, index_coupling+1]] ∪ S_space
S = Vector{Int}[]

vac = fockstate(b_fock, 0) ⊗ dagger(fockstate(b_fock, 0))
ρ0_space = [vac for i=1:Npoints]

psi_e = nlevelstate(b_atom, 2)
psi_g = nlevelstate(b_atom, 1)
psi = normalize(psi_e + psi_g)
ρ0_atom = psi ⊗ dagger(psi)

operators = [ρ0_atom; ρ0_space]
ρ0 = ce.CEOperator(operators, S);

T = [0:1.:5;]
tout, ρt_ce = ce.master(T, ρ0, H, J);

In [ ]:
for i_t=2:length(T)
    subplot(length(T), 1, i_t)
    plot(xpoints, [expect(i+1, n, ρt_ce[i_t]) for i=1:Npoints])
end

In [ ]:
plot(T, expect(1, psi_e⊗dagger(psi_e), ρt_ce))